In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\Jarian\AppData\Local\Temp\ipykernel_16912\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [8]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("../data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [11]:
# Your code
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

print(f"Train size:{train_data.size}, Test size:{test_data.size}")

Train size:1600, Test size:400


## Data Preprocessing

In [14]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [17]:
# Your code
# Import re module to use Regex for text cleaning
import re

# Function for cleaning HTML 
def clean_html(text):
    # Remove inline JavaScript
    text = re.sub(r'<script.*?>.*?</script>', '', text, flags=re.DOTALL)
    # Remove inline CSS
    text = re.sub(r'<style.*?>.*?</style>', '', text, flags=re.DOTALL)
    # Remove HTML comments
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    # Remove remaining HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Strip leading and trailing whitespace
    text = text.strip()
    return text
    
# Apply cleaning to the training and testing data
train_clean_html = train_data["text"].apply(clean_html)
test_clean_html = test_data["text"].apply(clean_html)

# Inspection of results for train data
print(train_clean_html.head())

29     ----------- REGARDS, MR NELSON SMITH.KINDLY RE...
535    I have not been able to reach oscar this am. W...
695    ; Huma Abedin B6I'm checking with Pat on the 5...
557    I can have it announced here on Monday - can't...
836    BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 San P...
Name: text, dtype: object


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [22]:
# Your code
def additional_cleaning(text):
    # Remove all special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove all single characters
    text = re.sub(r'\b\w\b', '', text)
    # Remove single characters from the start
    text = re.sub(r'^\s\w\s', '', text)
    # Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    # Remove prefixed 'b' (common in encoded text)
    text = re.sub(r'\bb\b', '', text)
    # Convert to lowercase
    text = text.lower()
    return text

# Apply additonal cleaning function to train and test data
train_cleaned = train_clean_html.apply(additional_cleaning)
test_cleaned = test_clean_html.apply(additional_cleaning)

# Inspect the cleaned data
print(train_cleaned.head())

29      regards mr nelson smithkindly reply me on my ...
535     have not been able to reach oscar this am we ...
695     huma abedin bim checking with pat on the will...
557      can have it announced here on monday cant today
836    bank of africaagence san pedro bp san pedro co...
Name: text, dtype: object


## Now let's work on removing stopwords
Remove the stopwords.

In [26]:
# Your code
# Initialize stop word set
stop_words = set(stopwords.words('english'))

train_cleaned = train_cleaned.apply(lambda text: ' '.join([word for word in text.split() if word not in stop_words]))
test_cleaned = test_cleaned.apply(lambda text: ' '.join([word for word in text.split() if word not in stop_words]))

# Inspect the cleaned data with stopwords removed
print(train_cleaned.head())

29     regards mr nelson smithkindly reply private em...
535           able reach oscar supposed send pdb receive
695    huma abedin bim checking pat work jack jake re...
557                          announced monday cant today
836    bank africaagence san pedro bp san pedro cote ...
Name: text, dtype: object


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [52]:
# Your code
from nltk.stem.wordnet import WordNetLemmatizer
#nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

# Lemmatize each word in the cleaned text
train_lemmatized = train_cleaned.apply(lambda text: ' '.join([lemmatizer.lemmatize(word) for word in text.split()]))
test_lemmatized = test_cleaned.apply(lambda text: ' '.join([lemmatizer.lemmatize(word) for word in text.split()]))

# Inspect the lemmatized data
print(train_lemmatized.head())

29     regard mr nelson smithkindly reply private ema...
535           able reach oscar supposed send pdb receive
695    huma abedin bim checking pat work jack jake re...
557                          announced monday cant today
836    bank africaagence san pedro bp san pedro cote ...
Name: text, dtype: object


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [54]:
# Your code
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

X = vectorizer.fit_transform(train_lemmatized)

# Show the Bag of Words feature names and the document-term matrix
print("Feature Names:", vectorizer.get_feature_names_out())
print("Document-Term Matrix:\n", X.toarray())

Feature Names: ['aac' 'aaclocated' 'aae' ... 'zumadirector' 'zumae' 'zurich']
Document-Term Matrix:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


## Extra features

In [58]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
# Define the regular expressions for money symbols and suspicious words
money_symbol_list = "|".join(["euro", "dollar", "pound", "€", "$"])
suspicious_words = "|".join(["free", "cheap", "sex", "money", "account", "bank", "fund", "transfer", "transaction", "win", "deposit", "password"])

# Adding the indicators and text length to the training data
train_lemmatized = train_lemmatized.apply(lambda x: x.lower())  # Make sure text is lowercase if not already
train_data['money_mark'] = train_lemmatized.str.contains(money_symbol_list)*1
train_data['suspicious_words'] = train_lemmatized.str.contains(suspicious_words)*1
train_data['text_len'] = train_lemmatized.apply(lambda x: len(x))

# Adding the indicators and text length to the test data
test_lemmatized = test_lemmatized.apply(lambda x: x.lower())  # Make sure text is lowercase if not already
test_data['money_mark'] = test_lemmatized.str.contains(money_symbol_list)*1
test_data['suspicious_words'] = test_lemmatized.str.contains(suspicious_words)*1
test_data['text_len'] = test_lemmatized.apply(lambda x: len(x))

# Inspect the training data
print(train_data.head())

                                                  text  label  money_mark  \
29   ----------- REGARDS, MR NELSON SMITH.KINDLY RE...      1           1   
535  I have not been able to reach oscar this am. W...      0           1   
695  ; Huma Abedin B6I'm checking with Pat on the 5...      0           1   
557  I can have it announced here on Monday - can't...      0           1   
836      BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...      1           1   

     suspicious_words  text_len  
29                  0        75  
535                 0        42  
695                 0        79  
557                 0        27  
836                 1      1067  


## How would work the Bag of Words with Count Vectorizer concept?

In [ ]:
# Your code

# The Bag of Words (BoW) model represents text as a collection of unique words, disregarding grammar and word order. 
# Using CountVectorizer from sklearn, we can convert a set of text documents into a Document-Term Matrix (DTM), where each row corresponds to a document and each column represents a unique word from the vocabulary. 
# The values in the matrix indicate the frequency of each word in the respective document. 
# After preprocessing the text (e.g., lemmatization, stopwords removal), we use CountVectorizer to tokenize the text and create the DTM. 
# This process converts the text data into a numerical format, making it suitable for machine learning models. 
# Additionally, parameters like stop_words, ngram_range, and max_features can be used to refine the model by excluding common words, including word sequences, or limiting the vocabulary size.

## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [62]:
# Your code
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the entire dataset (combine train and test datasets)
all_data = pd.concat([train_lemmatized, test_lemmatized])  # Concatenate train and test

# Transform all text data into TF-IDF features
X_tfidf = tfidf_vectorizer.fit_transform(all_data)

# Print the shape of the resulting TF-IDF matrix
print("Shape of the TF-IDF matrix:", X_tfidf.shape)

Shape of the TF-IDF matrix: (1000, 19349)


## And the Train a Classifier?

In [70]:
# Your code
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

# Initialize the Naive Bayes classifier
nb_classifier = MultinomialNB()

# Train the classifier on the TF-IDF transformed training data
nb_classifier.fit(X_train_tfidf, y_train)

# Make predictions on the test set
y_pred = nb_classifier.predict(X_test_tfidf)

# Evaluate the classifier's performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.54
Classification Report:
               precision    recall  f1-score   support

           0       0.55      0.94      0.69       110
           1       0.42      0.06      0.10        90

    accuracy                           0.54       200
   macro avg       0.48      0.50      0.39       200
weighted avg       0.49      0.54      0.42       200



### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
# Your code